In [1]:
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.llms.huggingface_pipeline import HuggingFacePipeline

device = "cuda" # 加载模型的设备

model = AutoModelForCausalLM.from_pretrained(
   "M:\model\Qwen1.5-1.8b",
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained("M:\model\Qwen1.5-1.8b")
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=500)
llm = HuggingFacePipeline(pipeline=pipe)

c:\Users\gu\AppData\Local\conda\conda\envs\py310\lib\site-packages\accelerate\utils\modeling.py:1365: UserWarning: Current model requires 738203136 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
prompt ="战国七雄，为什么秦只灭了六国？"
messages = [
    {"role": "system", "content": "你是一个有用的助手。"},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
response

In [9]:
from langchain_community.document_loaders import PDFPlumberLoader
file_path = '1关于数字化改革总体方案的汇报（省委改革办）.pdf'
loader = PDFPlumberLoader(file_path)
data = loader.load()


In [10]:
data[0]

Document(page_content='全 省 数 字 化 改 革 大 会\n会\n议\n资\n料\n（一）\n中共浙江省委全面深化改革委员会办公室\n年 月\n２０２１ ２\n书书书\n', metadata={'source': '1关于数字化改革总体方案的汇报（省委改革办）.pdf', 'file_path': '1关于数字化改革总体方案的汇报（省委改革办）.pdf', 'page': 0, 'total_pages': 44, 'CreationDate': "D:20210217220844+08'00'", 'Author': 'Founder Electronics Ltd', 'Creator': 'Founder PDF Library', 'Producer': 'Founder PDF Library', 'ModDate': "D:20210217213532+08'00'"})

In [11]:
# 文本分割
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, add_start_index=True)
all_splits = text_splitter.split_documents(data)

In [6]:
# 向量化存储
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "./bge-large-zh-v1.5"
model_kwargs = {"device": "cuda"}
encode_kwargs = {"normalize_embeddings": True}
bgeEmbeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

c:\Users\gu\AppData\Local\conda\conda\envs\py310\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [12]:
# 向量数据库
from langchain.vectorstores import FAISS
import pickle
vector = FAISS.from_documents(all_splits, bgeEmbeddings)
file = open('vector_1.8b.pkl', 'wb')
pickle.dump(vector, file)
file.close()


In [ ]:
import pickle
with open('vector_1.8b.pkl', 'rb') as file:
    vector = pickle.load(file)

In [14]:
retriever = vector.as_retriever(search_type="similarity", search_kwargs={"k": 3})
retriever.invoke("省委改革办数字化总体方案要求有哪些？")

[Document(page_content='全 省 数 字 化 改 革 大 会\n会\n议\n资\n料\n（一）\n中共浙江省委全面深化改革委员会办公室\n年 月\n２０２１ ２\n书书书', metadata={'source': '1关于数字化改革总体方案的汇报（省委改革办）.pdf', 'file_path': '1关于数字化改革总体方案的汇报（省委改革办）.pdf', 'page': 0, 'total_pages': 44, 'CreationDate': "D:20210217220844+08'00'", 'Author': 'Founder Electronics Ltd', 'Creator': 'Founder PDF Library', 'Producer': 'Founder PDF Library', 'ModDate': "D:20210217213532+08'00'", 'start_index': 0}),
 Document(page_content='关于数字化改革总体方案的汇报\n省委改革办\n根据会议安排，现就数字化改革总体方案（以下简称《总体方\n案》）有关情况汇报如下。\n一、起草背景\n省委十四届八次全会作出了以数字化改革为总抓手，撬动各\n领域各方面改革的决策部署。袁家军书记亲自谋划、亲自推动，在\n省委党校党的十九届五中全会精神集中轮训班上提出了“七个关\n键”的顶层设计，此后又分别于 年 月 日、 年 月\n２０２０ １２ ２９ ２０２１ １ ４\n日、 月 日、 月 日、 月 日、 月 日主持召开 次专题会\n１ ２２ ２ ７ ２ ９ ２ １４ ６\n议，听取汇报、作出指示，逐一研究明确数字化改革的定义内涵、目\n标任务、体系架构、方法路径、推进机制等重大问题，为数字化改革\n搭建了“四梁八柱”。郑栅洁省长多次强调，要充分发挥政府数字\n化转型先发优势，推进数字化改革，提高政府治理现代化水平。陈\n金彪秘书长、王昌荣书记、高兴夫副省长等省领导指导推动总体方\n案和相关系统方案起草工作。省委改革办、省大数据局切实履行\n牵头职责，会同省委办公厅、省政府办公厅、省委政法委、省发展改\n革委、省经信厅等单位认真研究，综合集成支撑平台方案和党政机\n关整体智治、数字政府、数字经济、数字社会、数字法治 个系统方\n５

In [15]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import BaseChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""仅根据所提供的上下文回答以下问题:

<context>
{context}
</context>

问题: {question}""")

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableSequence
retriever_chain: RunnableSequence = (
    {"context": retriever , "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [16]:
ans1 = retriever_chain.invoke("省委改革办数字化总体方案要求有哪些？")
# ans1 = retriever_chain.invoke("省委改革办数字化总体方案要求有哪些？")
print(ans1)


In [2]:
llm.invoke("战国七雄，秦国为什么只灭了六国")

'战国七雄，秦国为什么只灭了六国？\n\n秦国灭六国的原因有以下几点：\n\n1. 经济实力强大：秦国在战国时期，通过一系列的兼并战争，逐渐积累起强大的经济实力。秦国的铁器、牛耕技术、水利工程等都领先于其他诸侯国，这为秦国的扩张提供了强大的物质基础。此外，秦国的商业活动也非常活跃，通过商业贸易，秦国积累了大量的财富和影响力。\n\n2. 政治制度先进：秦国在秦孝公时期，推行了一系列的改革，包括废除分封制，实行郡县制，推行法治，建立中央集权制度等，这些改革都极大地提高了秦国的行政效率和治理能力，为秦国的统一奠定了基础。\n\n3. 军事力量强大：秦国在战国时期，拥有强大的军事力量，包括强大的步兵、骑兵、弓箭手等，这些军队在战争中表现出色，为秦国的统一提供了强大的军事支持。\n\n4. 人才储备丰富：秦国在战国时期，拥有大量的优秀人才，包括军事家、政治家、文学家等，这些人才为秦国的统一提供了强大的智力支持和思想支持。\n\n5. 政治策略灵活：秦国在统一六国的过程中，采取了灵活的策略，包括先攻后守、先弱后强、先弱后强等，这些策略使得秦国在统一的过程中能够有效地控制和利用各种资源，从而在战争中取得优势。\n\n综上所述，秦国灭六国的原因是多方面的，包括经济实力强大、政治制度先进、军事力量强大、人才储备丰富和政治策略灵活等。这些因素共同作用，使得秦国在战国时期能够迅速崛起，最终统一了六国。不断地学习和积累，是实现个人成长和事业成功的重要途径。以下是一些关于持续学习和积累的建议：\n\n1. 设定明确的学习目标：明确自己想要学习什么，是专业技能、知识还是个人兴趣。这将帮助你确定学习的方向和重点，从而更有针对性地进行学习。\n\n2. 制定学习计划：根据自己的学习目标，制定一份详细的学习计划，包括学习的时间、内容、方法等。这将帮助你更好地管理时间，提高学习效率。\n\n3. 多元化学习方式：学习的方式多种多样，包括阅读书籍、观看视频、参加研讨会、参加在线课程、实践操作等。选择适合自己的学习方式，可以提高学习的'